In [1]:
'''
Uncomment below section if running on sophia jupyter notebook
'''
import os
os.environ["HTTP_PROXY"]="proxy.alcf.anl.gov:3128"
os.environ["HTTPS_PROXY"]="proxy.alcf.anl.gov:3128"
os.environ["http_proxy"]="proxy.alcf.anl.gov:3128"
os.environ["https_proxy"]="proxy.alcf.anl.gov:3128"
os.environ["ftp_proxy"]="proxy.alcf.anl.gov:3128"

In [2]:
#!pip install transformers
#!pip install pandas
#!pip install torch
#!pip install tf-keras

In [5]:
#!pip install bertviz

# HW-Part 1

* Load in a generative model using the HuggingFace pipeline and generate text using a batch of prompts.
* Play with generative parameters such as temperature, max_new_tokens, and the model itself and explain the effect on the legibility of the model response. Try at least 4 different parameter/model combinations.
* Models that can be used include:
    * google/gemma-2-2b-it
    * microsoft/Phi-3-mini-4k-instruct
    * meta-llama/Llama-3.2-1B
    * Any model from this list: Text-generation models
    * gpt2 if having trouble loading these models in
* This guide should help! Text-generation strategies

In [2]:
import torch
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, AutoConfig
from transformers import pipeline

2024-11-25 23:54:06.535522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732578846.713602 3903203 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732578846.787310 3903203 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 23:54:07.240878: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
prompt_batch = ['Burning of fossil fuels',
                'Greenhouse gas emissions',
                'Global warming increases',
                'Renewable energy',
                'Implementing climate policies',
                'Warmer temperatures accelerate']

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)

# Basic test on text-generation
generator(prompt_batch, max_length=12, num_return_sequences=1)

/soft/applications/conda/2024-08-08/mconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[[{'generated_text': 'Burning of fossil fuels in the 1970s prompted a massive'}],
 [{'generated_text': 'Greenhouse gas emissions from the proposed Keystone XL pipeline. ('}],
 [{'generated_text': 'Global warming increases the rate of ocean acidification in the ocean'}],
 [{'generated_text': 'Renewable energy for human beings.\n\nThe only'}],
 [{'generated_text': 'Implementing climate policies\n\nThe international development community continues'}],
 [{'generated_text': 'Warmer temperatures accelerate the spread of solar radiation as a result'}]]

In [25]:
# increasing temperature to increase randomness
generator(prompt_batch, max_length=12, num_return_sequences=1, temperature=1.5)

[[{'generated_text': 'Burning of fossil fuels\n\nIn the end this can'}],
 [{'generated_text': 'Greenhouse gas emissions, the first round of research being carried'}],
 [{'generated_text': 'Global warming increases atmospheric GHB CO2 concentration as 0.'}],
 [{'generated_text': 'Renewable energy: $40.8bn on renewables'}],
 [{'generated_text': 'Implementing climate policies and a climate strategy is very difficult'}],
 [{'generated_text': 'Warmer temperatures accelerate on April 15 at about 12 PAC'}]]

In [26]:
# introducing sampling
generator(prompt_batch, max_length=12, num_return_sequences=1, do_sample=True, num_beams=5)

[[{'generated_text': 'Burning of fossil fuels is a threat to the environment,'}],
 [{'generated_text': 'Greenhouse gas emissions\n\nThe report, published by the'}],
 [{'generated_text': 'Global warming increases the risk of human-caused climate change'}],
 [{'generated_text': 'Renewable energy sources such as solar, wind and hydro'}],
 [{'generated_text': 'Implementing climate policies that are consistent with the goals of'}],
 [{'generated_text': "Warmer temperatures accelerate the warming of the Earth's surface,"}]]

In [35]:
# introducing sampling, beam search, early stopping for abrupt turncation
generator(prompt_batch, max_new_tokens=12, num_return_sequences=1, do_sample=True, 
          num_beams=5, early_stopping=True, repetition_penalty=3.5, length_penalty=0.5)

[[{'generated_text': 'Burning of fossil fuels in the U.S. is a major cause of climate'}],
 [{'generated_text': 'Greenhouse gas emissions from the U.S. are expected to increase by 1'}],
 [{'generated_text': 'Global warming increases the risk of extreme weather events, such as floods and d'}],
 [{'generated_text': 'Renewable energy sources such as wind, solar and geothermal are also being'}],
 [{'generated_text': 'Implementing climate policies that will reduce greenhouse gas emissions is a key part of the'}],
 [{'generated_text': 'Warmer temperatures accelerate in the South China Sea.\n\nChina and Japan have'}]]

In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)

generator(prompt_batch, max_length=12, num_return_sequences=1, temperature=1.5)

/soft/applications/conda/2024-08-08/mconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


[[{'generated_text': 'Burning of fossil fuels is a major contributor to climate'}],
 [{'generated_text': 'Greenhouse gas emissions from the transport sector are a major'}],
 [{'generated_text': 'Global warming increases the risk of heatwaves, study finds'}],
 [{'generated_text': "Renewable energy is the future.\nI'm"}],
 [{'generated_text': 'Implementing climate policies in the context of the CO'}],
 [{'generated_text': 'Warmer temperatures accelerate melting of Greenland ice sheet\n\n'}]]

In [45]:
generator(prompt_batch, max_new_tokens=12, num_return_sequences=1, do_sample=True, 
          num_beams=5, early_stopping=True, repetition_penalty=3.5, length_penalty=0.5)

[[{'generated_text': "Burning of fossil fuels is bad for the environment.  It's a fact."}],
 [{'generated_text': 'Greenhouse gas emissions are the main cause of global warming and climate change, but'}],
 [{'generated_text': 'Global warming increases the frequency of extreme heatwaves, study finds\nExtreme heat'}],
 [{'generated_text': 'Renewable energy is one of the most promising ways to fight climate change.'}],
 [{'generated_text': 'Implementing climate policies is not as simple as it sounds\nA new report from'}],
 [{'generated_text': 'Warmer temperatures accelerate wildfire danger in California\nLOS ANGELES (AP) —'}]]

## Comparison of models and parameters
* Model used:
    * gpt2
    * facebook/opt-1.3b
* Parameters checked:
    * do_sample: utilize multinomial sampling
    * num_beams: use beam search
    * early_stopping: reduce abrupt ending sentences
    * repetition_penalty: avoid repetition in text generation
    * length_penalty: included to reduce the length and reduce abrupt ending sentences
* Results:
    *  facebook/opt-1.3b generates better-turncated outputs than gpt2
    *  Beam-search multinomial sampling works better than greedy decoding for both models
    *  On increasing temperature, output randomness increases. However, it leads to abrupt sentence turncations

# HW-Part2 
* Load in 2 models of different parameter size (e.g. GPT2, meta-llama/Llama-2-7b-chat-hf, or distilbert/distilgpt2) and analyze the BertViz for each. How does the attention mechanisms change depending on model size?

In [41]:
from transformers import AutoTokenizer, AutoModel, utils, AutoModelForCausalLM

from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings


model_name = 'openai-community/gpt2'
input_text = "Greenhouse gas emissions"
model = AutoModelForCausalLM.from_pretrained(model_name, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
outputs = model(inputs)  # Run model
attention = outputs[-1]  # Retrieve attention from model outputs
tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings
model_view(attention, tokens)  # Display model view

<IPython.core.display.Javascript object>

In [42]:

model_name = 'facebook/opt-1.3b'
input_text = "Greenhouse gas emissions"
model = AutoModelForCausalLM.from_pretrained(model_name, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
outputs = model(inputs)  # Run model
attention = outputs[-1]  # Retrieve attention from model outputs
tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings
model_view(attention, tokens)  # Display model view

pytorch_model.bin:  98%|#########8| 2.58G/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

<IPython.core.display.Javascript object>

## Comparison of attention mechanism (gpt2 vs facebook/opt-1.3b)
* facebook/opt-1.3b has way more number of layers and heads than gpt2
* attentions look similar but denser for facebook/opt-1.3b